In [4]:
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.metrics import accuracy_score, precision_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import re


# Load the dataset
data = pd.read_csv('my dataset 6.csv')
#rremove punctuation
data['genre'] = data['genre'].apply(lambda x: re.sub(r'[^\w\s]', '', x)) 
# Split the dataset into a training set and a test set
train_data = data.sample(frac=0.8, random_state=42)
test_data = data.drop(train_data.index)

# Create a dataframe with song names and genres only
song_genre_df = data[['song name', 'genre']]

# Encode the genre column
le = LabelEncoder()
le.fit(data['genre'])
song_genre_df['genre'] = le.fit_transform(song_genre_df['genre'])

# Train the K-means model with k=3
kmeans = KMeans(n_clusters=3)
kmeans.fit(song_genre_df[['genre']])

# Take the input song name from the user
song_name = input("Enter a song name: ")

if song_name not in song_genre_df['song name'].values:
    print("Song not found in the database.")
else:
    # Get the genre of the input song
    song_genre = song_genre_df[song_genre_df['song name'] == song_name]['genre'].iloc[0]

    # Get the cluster label of the input song
    song_cluster_label = kmeans.predict([[song_genre]])[0]

    # Compute the distances between the input song and all songs in the same cluster
    song_distances = euclidean_distances(song_genre_df[kmeans.labels_ == song_cluster_label]['genre'].values.reshape(-1, 1), [[song_genre]])

    # Get the indices of the top 5 most similar songs
    top5_indices = song_distances.argsort(axis=0)[:5].flatten()

    # Get the recommended songs
    recommended_songs = song_genre_df[kmeans.labels_ == song_cluster_label]['song name'].iloc[top5_indices]

    # Print the recommended songs
    print("Top 5 recommended songs based on the genre of", song_name, ":")
    print(recommended_songs.tolist())
    

# Split the data into train and test sets
train_indices, test_indices = train_test_split(data.index, test_size=0.2, random_state=42)

# Fit a k-means clustering model to the training data
kmeans = KMeans(n_clusters=5, random_state=42)
#kmeans.fit(data.iloc[train_indices]['genre'])
kmeans.fit(song_genre_df[['genre']])

# Get the labels for the test data
#y_test = data.iloc[test_indices]['genre'].values
y_test = le.transform(data.iloc[test_indices]['genre'].values)
y_pred = kmeans.predict(y_test.reshape(-1, 1))
#y_pred = kmeans.predict(data.iloc[test_indices]['genre'].values.reshape(-1, 1))

# Compute the accuracy and precision of the model
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')

# Print the accuracy and precision
print("Accuracy: {:.2f}".format(accuracy))
print("Precision: {:.2f}".format(precision))



C:\Users\91910\AppData\Local\Temp\ipykernel_11892\109242743.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  song_genre_df['genre'] = le.fit_transform(song_genre_df['genre'])


Enter a song name: Sunno a buke
Top 5 recommended songs based on the genre of Sunno a buke :
['Chok chok gelo', 'Shiuli Phuler Mala Dole  ', 'Ebar Nabin Mantre Hobe  ', 'Aslo Jakhon Phuler Fagun ', 'Palas Phuler']
Accuracy: 0.69
Precision: 0.65


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KMeans was fitted with feature names
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KMeans was fitted with feature names
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
